# Sample code for Separate VGG16 Model

In [1]:
from typing import Any, cast, Dict, List, Optional, Union

import torch
import torch.nn as nn
import torchvision.models as models

import tensorflow as tf
import onnx2tf

2024-04-13 07:18:24.327521: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 07:18:24.327571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 07:18:24.328304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 07:18:24.332360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Separate VGG16 model top 5 layers

In [2]:
# Load VGG16 Model
vgg_model = models.vgg16(pretrained=True)

# Get state_dict of top 5 layers
top_layers_state_dict = {}
for i, (layer_name, param) in enumerate(vgg_model.named_parameters()):
    print(f'param #{i}: {layer_name}')
    if 'features' in layer_name and i < 5*2:
        top_layers_state_dict[layer_name] = param

# save file path
save_path = 'vgg_top5_layers.pth'

# save state_dict
torch.save(top_layers_state_dict, save_path)

param #0: features.0.weight
param #1: features.0.bias
param #2: features.2.weight
param #3: features.2.bias
param #4: features.5.weight
param #5: features.5.bias
param #6: features.7.weight
param #7: features.7.bias
param #8: features.10.weight
param #9: features.10.bias
param #10: features.12.weight
param #11: features.12.bias
param #12: features.14.weight
param #13: features.14.bias
param #14: features.17.weight
param #15: features.17.bias
param #16: features.19.weight
param #17: features.19.bias
param #18: features.21.weight
param #19: features.21.bias
param #20: features.24.weight
param #21: features.24.bias
param #22: features.26.weight
param #23: features.26.bias
param #24: features.28.weight
param #25: features.28.bias
param #26: classifier.0.weight
param #27: classifier.0.bias
param #28: classifier.3.weight
param #29: classifier.3.bias
param #30: classifier.6.weight
param #31: classifier.6.bias


In [3]:
# Copy and modify from https://pytorch.org/vision/main/_modules/torchvision/models/vgg.html#vgg16

__all__ = [
    "VGG",
    "VGG11_Weights",
    "VGG11_BN_Weights",
    "VGG13_Weights",
    "VGG13_BN_Weights",
    "VGG16_Weights",
    "VGG16_BN_Weights",
    "VGG19_Weights",
    "VGG19_BN_Weights",
    "vgg11",
    "vgg11_bn",
    "vgg13",
    "vgg13_bn",
    "vgg16",
    "vgg16_bn",
    "vgg19",
    "vgg19_bn",
]


class VGG(nn.Module):
    def __init__(
        self, features: nn.Module, num_classes: int = 1000, init_weights: bool = True, dropout: float = 0.5
    ) -> None:
        super().__init__()
        self.features = features
        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        return x


def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False, n_layers: int=100) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    feature_count = 0
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v

            feature_count += 1

        if (feature_count >= n_layers):
            break
    return nn.Sequential(*layers)


cfgs: Dict[str, List[Union[str, int]]] = {
    "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
}


def vgg_features(cfg: str, batch_norm: bool, progress: bool, n_layers: int, **kwargs: Any) -> VGG:
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm, n_layers=n_layers), **kwargs)
    return model

In [4]:
vgg_model_features = vgg_features("D", False, True, 5)
vgg_model_features

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
  )
)

In [5]:
vgg_model_features.load_state_dict(torch.load(save_path))

<All keys matched successfully>

In [6]:
vgg_model_features.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
  )
)

In [7]:
input_tensor = torch.randn(1, 3, 224, 224)

In [8]:
feature_tensor = vgg_model_features(input_tensor)
print(feature_tensor.shape)

torch.Size([1, 256, 56, 56])


In [9]:
feature_tensor_hook = None
def hook_fn(module, input, output):
    global feature_tensor_hook
    feature_tensor_hook = output

hook_layer = vgg_model.features[11]
hook_layer.register_forward_hook(hook_fn)

In [10]:
vgg_model.eval()
output_tensor = vgg_model(input_tensor)

In [11]:
feature_tensor_hook

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [15.4093,  6.7857,  4.8907,  ...,  0.9767,  0.0735,  0.0000],
          [15.0378, 14.3292,  5.7194,  ...,  2.0751,  0.0000,  0.0000],
          ...,
          [11.7255,  0.0000,  2.6770,  ...,  4.2560,  0.0000,  0.0000],
          [11.3627,  0.0000,  0.0000,  ...,  1.4773,  0.0000,  0.0000],
          [ 9.7847,  4.7043,  5.4600,  ..., 11.3073, 11.9892,  0.3918]],

         [[ 6.0661, 11.0409,  4.5133,  ..., 11.6577, 13.2693, 21.1984],
          [ 9.4763,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 3.7147,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 2.0520,  3.4882,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.2848,  0.1977],
          [ 4.5651,  0.0000,  8.1412,  ...,  3.0580, 10.4078,  3.7552]],

         [[ 0.0000,  2.2535,  2.6899,  ...,  3.1037,  4.2589,  0.0000],
          [ 0.0000,  5.2941,  

In [12]:
feature_tensor

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [15.4093,  6.7857,  4.8907,  ...,  0.9767,  0.0735,  0.0000],
          [15.0378, 14.3292,  5.7194,  ...,  2.0751,  0.0000,  0.0000],
          ...,
          [11.7255,  0.0000,  2.6770,  ...,  4.2560,  0.0000,  0.0000],
          [11.3627,  0.0000,  0.0000,  ...,  1.4773,  0.0000,  0.0000],
          [ 9.7847,  4.7043,  5.4600,  ..., 11.3073, 11.9892,  0.3918]],

         [[ 6.0661, 11.0409,  4.5133,  ..., 11.6577, 13.2693, 21.1984],
          [ 9.4763,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 3.7147,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 2.0520,  3.4882,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.2848,  0.1977],
          [ 4.5651,  0.0000,  8.1412,  ...,  3.0580, 10.4078,  3.7552]],

         [[ 0.0000,  2.2535,  2.6899,  ...,  3.1037,  4.2589,  0.0000],
          [ 0.0000,  5.2941,  

In [13]:
(feature_tensor_hook == feature_tensor).all()

tensor(True)

## Convert pth to tflite

In [14]:
# --- pth to onnx ---
torch.onnx.export(vgg_model_features, input_tensor, 'vgg_top5_layers.onnx', verbose=True)

!onnx2tf -i vgg_top5_layers.onnx -oh5

Exported graph: graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %features.0.weight : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %features.0.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %features.2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.5.weight : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %features.5.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %features.7.weight : Float(128, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %features.7.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %features.10.weight : Float(256, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %features.10.bias : Float(256, strides=[1], require

## Inference dummy data using TFLite

In [15]:
# --- Load tflite ---
interpreter = tf.lite.Interpreter(model_path='saved_model/vgg_top5_layers_float32.tflite')
interpreter.allocate_tensors()

# --- Load index ---
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

# --- Set input tensor ---
interpreter.set_tensor(input_index, input_tensor.numpy().transpose([0, 2, 3, 1]))

# --- Inference ---
interpreter.invoke()

# --- Get output tensor ---
output_tensor_tflite = interpreter.get_tensor(output_index)
output_tensor_tflite = output_tensor_tflite.transpose([0, 3, 1, 2])
output_tensor_tflite

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


array([[[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [15.409332  ,  6.78569   ,  4.890681  , ...,  0.97668546,
           0.07349937,  0.        ],
         [15.037783  , 14.329218  ,  5.71934   , ...,  2.0750613 ,
           0.        ,  0.        ],
         ...,
         [11.7255125 ,  0.        ,  2.67697   , ...,  4.2560186 ,
           0.        ,  0.        ],
         [11.362651  ,  0.        ,  0.        , ...,  1.4772967 ,
           0.        ,  0.        ],
         [ 9.784675  ,  4.7042713 ,  5.4599843 , ..., 11.30734   ,
          11.989197  ,  0.3917982 ]],

        [[ 6.0660524 , 11.0409    ,  4.513326  , ..., 11.6576605 ,
          13.269262  , 21.198433  ],
         [ 9.476288  ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 3.7147267 ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 2.0520318 ,  3.4882505 

In [16]:
diff = feature_tensor.detach().numpy() - output_tensor_tflite

In [17]:
diff.min()

-8.392334e-05

In [18]:
diff.max()

8.583069e-05